# 퍼셉트론(Perceptron)

퍼셉트론은 다수의 신호를 입력으로 받아 하나의 신호를 출력한다.<br>
퍼셉트론은 입출력을 갖춘 알고리즘이다. 입력을 주면 정해진 규칙에 따른 값을 출력한다.

<b>수식</b>   
$
 y =\begin{cases}0 (w_{1}x_{1} + w_{2}w_{2} \leq  \theta )\\1 (w_{1}x_{1} + w_{2}w_{2} > \theta )\end{cases}
$

### 1. 논리 회로

#### 1) AND 게이트 퍼셉트론

In [2]:
import numpy as np

def AND(x1: int, x2: int) -> bool:
    # 가중치를 곱하고 더하여 편향을 더했을 때 0보다 작거나 같으면 False 반환
    x = np.array([x1, x2])
    w = np.array([0.5, 0.5])
    b = -0.7
    sum = np.sum(x * w) + b
    if sum <= 0:
        return False
    else:
        return True
    
print(AND(0,0))
print(AND(1,0))
print(AND(0,1))
print(AND(1,1))

False
False
False
True


### 2) OR 게이트 퍼셉트론

In [4]:
import numpy as np

def OR(x1: int, x2: int) -> bool:
    x = np.array([x1, x2])
    w = np.array([0.5, 0.5])
    b = -0.3
    sum = np.sum(x * w) + b
    if sum <= 0:
        return False
    else:
        return True
    
print(OR(0,0))
print(OR(1,0))
print(OR(0,1))
print(OR(1,1))

False
True
True
True


### 3) NAND 게이트 퍼셉트론

In [7]:
import numpy as np

def NAND(x1: int, x2: int) -> bool:
    x = np.array([x1, x2])
    w = np.array([-0.5, -0.5])
    b = 0.7
    sum = np.sum(x * w) + b
    if sum <= 0:
        return False
    else:
        return True
    
print(NAND(0,0))
print(NAND(1,0))
print(NAND(0,1))
print(NAND(1,1))

True
True
True
False


### 4) XOR 게이트 다층 퍼셉트론

단층 퍼셉트론만으로는 선형적인 표현만 가능하고, 비선형적인 XOR을 표현할 수 없다.<br>
이를 해결하기 위해 퍼셉트론을 여러 층으로 구성해 비선형적인 표현을 가능하게 할 수 있다.

In [8]:
def XOR(x1: int, x2: int) -> bool:
    s1 = OR(x1, x2)
    s2 = NAND(x1, x2)
    return AND(s1, s2)

print(XOR(0,0))
print(XOR(1,0))
print(XOR(0,1))
print(XOR(1,1))

False
True
True
False


In [79]:
def AND(a: int, b: int, bias: float) -> int:
    return 1 if a + b > bias else 0
    
def NAND(a: int, b: int, bias: float) -> int:
    return 0 if AND(a=a, b=b, bias=bias) == 1 else 1

def OR(a: int, b: int, bias: float=None) -> int:
    bias = 0 if bias is None else bias
    return 1 if a + b > bias else bias

def XOR(a: int, b: int, bias: float) -> int:
    _or = OR(a=a, b=b, bias=0)
    nand = NAND(a=a, b=b, bias=bias)
    return AND(a=_or, b=nand, bias=bias)

base_bias=1.5

input_a = 1
input_b = 1

xor = XOR(a=input_a, b=input_b, bias=1.5)

print(f"a({input_a}) + b({input_b}) = {xor}")

# Half Adder
def half_add(a: int, b: int) -> dict[str, int]: 
    s = XOR(a=a, b=b, bias=1.5)
    c_out = AND(a=a, b=b, bias=1.5)
    return { "S": s, "C_OUT": c_out }

result_half_add = half_add(a=input_a, b=input_b)

print(f"a({input_a}) + b({input_b}) = {result_half_add}")

# Full Adder
def full_add(a: int, b: int, c_in: int) -> dict[str, int]:
    p = XOR(a=a, b=b, bias=1.5)
    q = AND(a=p, b=c_in, bias=1.5)
    r = AND(a=a, b=b, bias=1.5)
    s = XOR(a=p, b=c_in, bias=1.5)
    c_out = OR(a=q, b=r, bias=0)
    return { "S": s, "C_OUT": c_out }

input_c_in = 0
result_full_add = full_add(a=input_a, b=input_b, c_in=input_c_in)

print(f"a({input_a}) + b({input_b}) + c_in({input_c_in}) = {result_full_add}")

from functools import reduce

# bit adder
def bit_adder(a: str, b: str) -> int:
    a_bits = [ int(bit) for bit in a ]
    b_bits = [ int(bit) for bit in b ]
    len_bits = len(a)
    sum_bits = [ 0 for _ in range(len_bits+1)]
    
    # half add
    half_dict = half_add(a=a_bits[len_bits-1], b=b_bits[len_bits-1])
    sum_bits[0] = half_dict["S"]
    c_out = half_dict["C_OUT"]
    
    # full add
    for i in range(len_bits-2, -1, -1):
        full_dict = full_add(a=a_bits[i], b=b_bits[i], c_in=c_out)
        s, c_out = full_dict["S"], full_dict["C_OUT"]
        sum_bits[len_bits -i -1] = s
    sum_bits[len_bits] = c_out
    
    return reduce(lambda acc, enum: acc + enum[1]*2**(enum[0]), 
                    enumerate(sum_bits), 
                    0)

input_bit_adder_a = "0011"
input_bit_adder_b = "0111"
print(f"a({input_bit_adder_a}) + b({input_bit_adder_b}) = {bit_adder(input_bit_adder_a, input_bit_adder_b)}")

a(1) + b(1) = 0
a(1) + b(1) = {'S': 0, 'C_OUT': 1}
a(1) + b(1) + c_in(0) = {'S': 0, 'C_OUT': 1}
a(0011) + b(0111) = 10
